# Análise do Risco Geo-Hidrológico e Acidentes

https://www.gov.br/cemaden/pt-br/assuntos/riscos-geo-hidrologicos


In [19]:
# Importing Dependencies
import geopandas as gpd
import fiona
import scipy.stats as stats
from shapely.geometry import Point
import pandas as pd
from datetime import datetime

fiona.drvsupport.supported_drivers["KML"] = "rw"

In [20]:
# Read KML File from December Hidrologic Risk
december_map = gpd.read_file("mapas/Dezembro.kml", driver="KML")
december_map

,Name,Description,geometry
0,Dezembro,,"POLYGON Z ((-44.61194 -23.37628 0.00000, -41.9..."


In [21]:
# Read KML File from January Hidrologic Risk
january_map = gpd.read_file("mapas/Janeiro.kml", driver="KML")
january_map

,Name,Description,geometry
0,Janeiro2,,"POLYGON Z ((-43.62187 -23.07444 0.00000, -42.6..."


In [22]:
# Read KML File from February Hidrologic Risk
february_map = gpd.read_file("mapas/Fevereiro.kml", driver="KML")
february_map

,Name,Description,geometry
0,Fevereiro,,"POLYGON Z ((-47.83853 -25.29331 0.00000, -47.3..."


In [23]:
# Read DataTran Base from 2022
data = pd.read_csv("raw_data/datatran_2022.csv")
data.head()

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
0,405151,1/1/2022,sábado,1:35:00,PI,316.0,415.0,MARCOLANDIA,Ingestão de álcool pelo condutor,Colisão traseira,...,0,1,0,1,2,-7.432800,-40.682619,SPRF-PI,DEL04-PI,UOP03-DEL04-PI
1,405158,1/1/2022,sábado,2:40:00,PR,116.0,33.0,CAMPINA GRANDE DO SUL,Ingestão de álcool pelo condutor,Tombamento,...,0,0,0,1,1,-25.114403,-48.846755,SPRF-PR,DEL01-PR,UOP02-DEL01-PR
2,405172,1/1/2022,sábado,5:22:00,MS,163.0,393.0,NOVA ALVORADA DO SUL,Condutor deixou de manter distância do veículo...,Colisão traseira,...,0,1,0,1,2,-21.228445,-54.456296,SPRF-MS,DEL02-MS,UOP01-DEL02-MS
3,405203,1/1/2022,sábado,7:00:00,RJ,101.0,457.0,ANGRA DOS REIS,Reação tardia ou ineficiente do condutor,Colisão frontal,...,1,1,0,1,2,-23.031498,-44.177153,SPRF-RJ,DEL03-RJ,UOP02-DEL03-RJ
4,405207,1/1/2022,sábado,9:00:00,MG,40.0,508.3,RIBEIRAO DAS NEVES,Acumulo de água sobre o pavimento,Saída de leito carroçável,...,0,3,0,0,1,-19.760612,-44.134754,SPRF-MG,DEL02-MG,UOP01-DEL02-MG


In [24]:
# Collecting month from the accident
def get_month(row: pd.Series):
    # Concatenando data e hora do acidente
    date: datetime = datetime.strptime(row["data_inversa"], "%m/%d/%Y")
    return date.month


data["month"] = data.apply(get_month, axis=1)

In [25]:
# Capturing Geo information from accidents in December
december_accidents = data.loc[data["month"] == 12][
    ["uf", "longitude", "latitude", "causa_acidente"]
]
december_accidents

,uf,longitude,latitude,causa_acidente
15196,SP,-47.062412,-23.939013,Reação tardia ou ineficiente do condutor
15197,RJ,-43.110465,-22.869811,Carga excessiva e/ou mal acondicionada
15200,PE,-38.912544,-8.008322,Transitar na contramão
15201,SP,-45.545171,-23.028713,Condutor deixou de manter distância do veículo...
15204,ES,-40.876088,-20.859236,Velocidade Incompatível
...,...,...,...,...
64523,MG,-45.244994,-21.554138,Velocidade Incompatível
64525,RJ,-43.810790,-22.842106,Ausência de reação do condutor
64531,RS,-51.175930,-29.896475,Ultrapassagem Indevida
64532,MG,-41.853610,-20.253033,Acessar a via sem observar a presença dos outr...


In [26]:
# Capturing Geo information from accidents in January
jan_accidents = data.loc[data["month"] == 1][
    ["uf", "longitude", "latitude", "causa_acidente"]
]
jan_accidents

,uf,longitude,latitude,causa_acidente
0,PI,-40.682619,-7.432800,Ingestão de álcool pelo condutor
1,PR,-48.846755,-25.114403,Ingestão de álcool pelo condutor
2,MS,-54.456296,-21.228445,Condutor deixou de manter distância do veículo...
3,RJ,-44.177153,-23.031498,Reação tardia ou ineficiente do condutor
4,MG,-44.134754,-19.760612,Acumulo de água sobre o pavimento
...,...,...,...,...
61541,AL,-35.842455,-8.950803,Animais na Pista
61957,MG,-44.966878,-17.381781,Velocidade Incompatível
64240,PE,-34.931767,-7.983443,Acesso irregular
64444,MG,-43.210153,-19.849754,Pista esburacada


In [27]:
# Capturing Geo information from accidents in February
february_accidents = data.loc[data["month"] == 2][
    ["uf", "longitude", "latitude", "causa_acidente"]
]
february_accidents

,uf,longitude,latitude,causa_acidente
1219,SC,-48.630032,-27.534137,Velocidade Incompatível
1220,PR,-50.070922,-25.201851,Condutor Dormindo
1223,BA,-39.217213,-12.865363,Condutor deixou de manter distância do veículo...
1224,RJ,-43.285795,-22.628979,Acessar a via sem observar a presença dos outr...
1225,ES,-40.458741,-20.382507,Condutor deixou de manter distância do veículo...
...,...,...,...,...
64363,PE,-35.069899,-8.302428,Reação tardia ou ineficiente do condutor
64417,BA,-39.495742,-13.865154,Reação tardia ou ineficiente do condutor
64443,RJ,-44.191761,-22.541349,Condutor deixou de manter distância do veículo...
64472,MG,-44.044366,-19.908832,Acessar a via sem observar a presença dos outr...


In [28]:
# Registring accidents location as Geometric Points
geometic_points_dec = []
geometic_points_jan = []
geometic_points_feb = []


for xy in zip(december_accidents["longitude"], december_accidents["latitude"]):
    geometic_points_dec.append(Point(xy))

accident_locations_dec = gpd.GeoDataFrame(
    december_accidents, geometry=geometic_points_dec
)
accident_locations_dec.crs = "epsg:4326"
accident_locations_dec["in_my_map"] = False


for xy in zip(jan_accidents["longitude"], jan_accidents["latitude"]):
    geometic_points_jan.append(Point(xy))

accident_locations_jan = gpd.GeoDataFrame(jan_accidents, geometry=geometic_points_jan)
accident_locations_jan.crs = "epsg:4326"
accident_locations_jan["in_my_map"] = False

for xy in zip(december_accidents["longitude"], december_accidents["latitude"]):
    geometic_points_feb.append(Point(xy))

accident_locations_feb = gpd.GeoDataFrame(
    december_accidents, geometry=geometic_points_feb
)
accident_locations_feb.crs = "epsg:4326"
accident_locations_feb["in_my_map"] = False

In [29]:
# Checking if accident is inside Risk Zone (December)
for i in accident_locations_dec.index:
    point = accident_locations_dec.geometry.loc[i]
    accident_locations_dec.loc[i, "in_my_map"] = december_map.contains(point).values

accident_locations_dec.head()

,uf,longitude,latitude,causa_acidente,geometry,in_my_map
15196,SP,-47.062412,-23.939013,Reação tardia ou ineficiente do condutor,POINT (-47.06241 -23.93901),False
15197,RJ,-43.110465,-22.869811,Carga excessiva e/ou mal acondicionada,POINT (-43.11046 -22.86981),True
15200,PE,-38.912544,-8.008322,Transitar na contramão,POINT (-38.91254 -8.00832),False
15201,SP,-45.545171,-23.028713,Condutor deixou de manter distância do veículo...,POINT (-45.54517 -23.02871),False
15204,ES,-40.876088,-20.859236,Velocidade Incompatível,POINT (-40.87609 -20.85924),True


In [30]:
# Checking if accident is inside Risk Zone (January)
for i in accident_locations_jan.index:
    point = accident_locations_jan.geometry.loc[i]
    accident_locations_jan.loc[i, "in_my_map"] = january_map.contains(point).values

accident_locations_jan.head()

,uf,longitude,latitude,causa_acidente,geometry,in_my_map
0,PI,-40.682619,-7.432800,Ingestão de álcool pelo condutor,POINT (-40.68262 -7.43280),True
1,PR,-48.846755,-25.114403,Ingestão de álcool pelo condutor,POINT (-48.84676 -25.11440),False
2,MS,-54.456296,-21.228445,Condutor deixou de manter distância do veículo...,POINT (-54.45630 -21.22845),False
3,RJ,-44.177153,-23.031498,Reação tardia ou ineficiente do condutor,POINT (-44.17715 -23.03150),True
4,MG,-44.134754,-19.760612,Acumulo de água sobre o pavimento,POINT (-44.13475 -19.76061),True


In [31]:
# Checking if accident is inside Risk Zone (February)
for i in accident_locations_feb.index:
    point = accident_locations_feb.geometry.loc[i]
    accident_locations_feb.loc[i, "in_my_map"] = february_map.contains(point).values

accident_locations_feb.head()

,uf,longitude,latitude,causa_acidente,geometry,in_my_map
15196,SP,-47.062412,-23.939013,Reação tardia ou ineficiente do condutor,POINT (-47.06241 -23.93901),True
15197,RJ,-43.110465,-22.869811,Carga excessiva e/ou mal acondicionada,POINT (-43.11046 -22.86981),True
15200,PE,-38.912544,-8.008322,Transitar na contramão,POINT (-38.91254 -8.00832),False
15201,SP,-45.545171,-23.028713,Condutor deixou de manter distância do veículo...,POINT (-45.54517 -23.02871),True
15204,ES,-40.876088,-20.859236,Velocidade Incompatível,POINT (-40.87609 -20.85924),False


In [32]:
# Collecting accidents directly caused by rain
rain_accidents_dec = pd.concat(
    [
        accident_locations_dec[accident_locations_dec.causa_acidente == "Chuva"],
        accident_locations_dec[
            accident_locations_dec.causa_acidente == "Acumulo de água sobre o pavimento"
        ],
    ]
)

rain_accidents_jan = pd.concat(
    [
        accident_locations_jan[accident_locations_jan.causa_acidente == "Chuva"],
        accident_locations_jan[
            accident_locations_jan.causa_acidente == "Acumulo de água sobre o pavimento"
        ],
    ]
)

rain_accidents_feb = pd.concat(
    [
        accident_locations_feb[accident_locations_feb.causa_acidente == "Chuva"],
        accident_locations_feb[
            accident_locations_feb.causa_acidente == "Acumulo de água sobre o pavimento"
        ],
    ]
)

In [33]:
# December - Rain Accidents
total_dec = len(rain_accidents_dec)
accidents_from_risk_dec = sum(rain_accidents_dec.in_my_map)

# January - Rain Accidents
total_jan = len(rain_accidents_jan)
accidents_from_risk_jan = sum(rain_accidents_jan.in_my_map)

# February - Rain Accidents
total_feb = len(rain_accidents_feb)
accidents_from_risk_feb = sum(rain_accidents_feb.in_my_map)

In [34]:
# Create Dataframe from rain accidents from risk region data
rain_accidents_from_risk = pd.DataFrame.from_dict(
    {
        "Dezembro": [
            accidents_from_risk_dec,
            total_dec - accidents_from_risk_dec,
            total_dec,
        ],
        "Janeiro": [
            accidents_from_risk_jan,
            total_jan - accidents_from_risk_jan,
            total_jan,
        ],
        "Fevereiro": [
            accidents_from_risk_feb,
            total_feb - accidents_from_risk_feb,
            total_feb,
        ],
    },
    orient="index",
    columns=[
        "Acidentes Dentro Região de Risco",
        "Acidentes Fora da Região de Risco",
        "Total",
    ],
)

rain_accidents_from_risk

,Acidentes Dentro Região de Risco,Acidentes Fora da Região de Risco,Total
Dezembro,70,157,227
Janeiro,112,134,246
Fevereiro,82,145,227


In [35]:
# December - All Accidents
total_dec = len(accident_locations_dec)
accidents_from_risk_dec = sum(accident_locations_dec.in_my_map)

# January - All Accidents
total_jan = len(accident_locations_jan)
accidents_from_risk_jan = sum(accident_locations_jan.in_my_map)

# February - All Accidents
total_feb = len(accident_locations_feb)
accidents_from_risk_feb = sum(accident_locations_feb.in_my_map)

In [36]:
# Create Dataframe from all accidents from risk region data
all_accidents_from_risk = pd.DataFrame.from_dict(
    {
        "Dezembro": [
            accidents_from_risk_dec,
            total_dec - accidents_from_risk_dec,
            total_dec,
        ],
        "Janeiro": [
            accidents_from_risk_jan,
            total_jan - accidents_from_risk_jan,
            total_jan,
        ],
        "Fevereiro": [
            accidents_from_risk_feb,
            total_feb - accidents_from_risk_feb,
            total_feb,
        ],
    },
    orient="index",
    columns=[
        "Acidentes Dentro Região de Risco",
        "Acidentes Fora da Região de Risco",
        "Total",
    ],
)

all_accidents_from_risk

,Acidentes Dentro Região de Risco,Acidentes Fora da Região de Risco,Total
Dezembro,1218,4604,5822
Janeiro,1724,3323,5047
Fevereiro,1323,4499,5822
